In [1]:
import pandas as pd
from pathlib import Path
from datetime import timedelta, datetime
import re

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def load_raw_data(csv_path):
    df = pd.read_csv(csv_path,
                    dtype={"starttime": "string", "stoptime": "string"},
                    keep_default_na=False,)
    return df

In [3]:
def parse_bike_time(series: pd.Series, freq: str = "h") -> pd.Series:
    s = series.astype(str).str.strip().str.replace("\u200b", "", regex=False)
    dt = pd.to_datetime(s, errors="coerce")
    # fix single-digit hour " 1:" -> " 01:" if needed
    m = dt.isna()
    if m.any():
        s2 = s[m].str.replace(r" (\d):", lambda x: f" 0{x.group(1)}:", regex=True)
        dt.loc[m] = pd.to_datetime(s2, format="%Y-%m-%d %H:%M:%S", errors="coerce")
    return dt.dt.floor(freq)

def normalize_station_id(s: pd.Series) -> pd.Series:
    # handles numeric strings / floats cleanly; keeps <NA> if missing
    return pd.to_numeric(s, errors="coerce").astype("Int64")

In [4]:
def transform_data(df: pd.DataFrame) -> pd.DataFrame:
    # ---------- Normalize timestamps to the hour ----------
    df["starttime"] = parse_bike_time(df["starttime"], freq="h")
    df["stoptime"]  = parse_bike_time(df["stoptime"],  freq="h")

    # ---------- Organize location data ----------
    start_meta = (
        df.rename(columns={
            "start station id": "station_id",
            "start station name": "station_name",
            "start station latitude": "latitude",
            "start station longitude": "longitude",
        })[["station_id", "station_name", "latitude", "longitude"]]
    )
    end_meta = (
        df.rename(columns={
            "end station id": "station_id",
            "end station name": "station_name",
            "end station latitude": "latitude",
            "end station longitude": "longitude",
        })[["station_id", "station_name", "latitude", "longitude"]]
    )

    stations = pd.concat([start_meta, end_meta], ignore_index=True)
    stations["station_id"] = normalize_station_id(stations["station_id"])
    stations = stations.dropna(subset=["station_id"]).drop_duplicates("station_id")

    # ---------- Count OUTS (undocks) by hour & start station ----------
    outs = (
        df.dropna(subset=["start station id"])
          .assign(station_id=lambda x: normalize_station_id(x["start station id"]))
          .groupby(["starttime", "station_id"], as_index=False)
          .size()
          .rename(columns={"starttime": "timestart", "size": "out"})
    )

    # ---------- Count INS (docks) by hour & end station ----------
    ins = (
        df.dropna(subset=["end station id"])
          .assign(station_id=lambda x: normalize_station_id(x["end station id"]))
          .groupby(["stoptime", "station_id"], as_index=False)
          .size()
          .rename(columns={"stoptime": "timestart", "size": "in"})
    )

    # ---------- Combine (outer join so hours with only in OR only out are kept) ----------
    hourly = (
        pd.merge(outs, ins, on=["timestart", "station_id"], how="outer")
          .fillna({"in": 0, "out": 0})
    )

    # ---------- (Optional) Complete grid: every station × every hour ----------
    start_hour = min(df["starttime"].min(), df["stoptime"].min())
    end_hour   = max(df["starttime"].max(), df["stoptime"].max())
    all_hours  = pd.date_range(start=start_hour, end=end_hour, freq="h")

    all_station_ids = (
        stations["station_id"]
        .dropna()
        .astype("int64")
        .unique()
    )

    full_idx = pd.MultiIndex.from_product(
        [all_hours, all_station_ids],
        names=["timestart", "station_id"]
    )

    hourly = (
        hourly.set_index(["timestart", "station_id"])
              .reindex(full_idx, fill_value=0)
              .reset_index()
    )

    hourly["in"]  = hourly["in"].astype("int64")
    hourly["out"] = hourly["out"].astype("int64")

    # ---------- Attach metadata & compute timeend ----------
    hourly = (
        hourly.merge(stations, on="station_id", how="left")
              .assign(timeend=lambda x: x["timestart"] + pd.Timedelta(hours=1))
    )

    # ---------- Add time-based features from the interval ----------
    # Hour-of-week: Monday=0 ... Sunday=6, so 0–167
    start_how = (hourly["timestart"].dt.dayofweek * 24 + hourly["timestart"].dt.hour).astype("int16")
    end_how   = (hourly["timeend"].dt.dayofweek   * 24 + hourly["timeend"].dt.hour).astype("int16")
    is_weekend = (hourly["timestart"].dt.dayofweek >= 5).astype("int8")  # Sat/Sun = 1
    month_num  = hourly["timestart"].dt.month.astype("int8")             # 1–12

    hourly = hourly.assign(
        start_hour_of_week=start_how,
        end_hour_of_week=end_how,
        is_weekend=is_weekend,
        month=month_num,
    )

    # ---------- Final column order ----------
    hourly = (
        hourly.loc[:, [
            "timestart", "timeend",
            "station_name",                 # ← moved here
            "start_hour_of_week", "end_hour_of_week", "is_weekend", "month",
            "station_id", "latitude", "longitude",
            "in", "out"
        ]]
        .sort_values(["timestart", "station_id"])
        .reset_index(drop=True)
    )

    return hourly


In [5]:
# from pathlib import Path
# from datetime import datetime
# import pandas as pd

# # ---------- Inputs ----------
# input_dir  = Path("/content/drive/MyDrive/Side hustles/DS+X/Data/Raw")
# output_dir = Path("/content/drive/MyDrive/Side hustles/DS+X/Data/Transformed")

# files = [
#     "202205-bluebikes-tripdata",
#     "202206-bluebikes-tripdata",
#     "202207-bluebikes-tripdata",
# ]

# paths = [input_dir / f"{stem}.csv" for stem in files]

# # ---------- Load & concatenate ----------
# # Assumes you already have load_raw_data(path) -> DataFrame
# # If schemas differ slightly, pd.concat(..., sort=False) keeps all columns.
# dfs = [load_raw_data(p) for p in paths]
# df_all = pd.concat(dfs, ignore_index=True, sort=False)

# # Optional sanity check
# print(f"Loaded {len(paths)} files; total rows: {len(df_all):,}")

# # ---------- Transform ----------
# transformed_data = transform_data(df_all)

# # ---------- Save (timestamped, safe filename) ----------
# ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")
# combo_tag = f"{files[0]}_to_{files[-1]}"  # e.g., 202205-..._to_202207-...
# out_path = output_dir / f"{combo_tag}_parsed_{ts}.csv"

# transformed_data.to_csv(out_path, index=False, date_format="%Y-%m-%d %H:%M:%S")
# print(f"Saved: {out_path}")


In [7]:
!pip -q install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.6 MB/s eta 0:00:00


In [20]:
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# --- Load ---
trainingPath = Path("/content/drive/MyDrive/Side hustles/DS+X/Data/Cleaned")
dtypes = {
    "stationid": "Int32",
    "month": "Int16",
    "weekhour": "Int16",
    "TMAX": "float32",
    "TMIN": "float32",
    "PRCP": "float32",
}
df = pd.read_csv(trainingPath / "2023.csv", dtype=dtypes, low_memory=False)

# --- Features / Targets ---
feature_cols = ["stationid", "month", "weekhour"]
target_cols = ["next_bike_in", "next_bike_out"]

# Ensure targets are numeric and drop rows with missing targets
df[target_cols] = df[target_cols].apply(pd.to_numeric, errors="coerce")
df = df.dropna(subset=target_cols).copy()

X = df[feature_cols]
y = df[target_cols]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, shuffle=True, stratify=df["stationid"]
)
print("-----Training Data-----")
print(X_train.head(100))
print(y_train.head(100))

print("-----Testing Data-----")
print(X_test.head(100))
print(y_test.head(100))

# --- Model ---
pre = "passthrough"  # <- no OHE, avoids missing column issues

xgb = XGBRegressor(
    objective="reg:squarederror",
    tree_method="hist",
    n_estimators=600,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
)

model = Pipeline([
    ("prep", pre),
    ("reg", MultiOutputRegressor(xgb, n_jobs=-1)),
])

# --- Train ---
model.fit(X_train, y_train)

# --- Evaluate ---
pred = model.predict(X_test)
pred_in, pred_out = pred[:, 0], pred[:, 1]

mae_in  = mean_absolute_error(y_test["next_bike_in"],  pred_in)
mae_out = mean_absolute_error(y_test["next_bike_out"], pred_out)
rmse_in = np.sqrt(mean_squared_error(y_test["next_bike_in"],  pred_in))
rmse_out= np.sqrt(mean_squared_error(y_test["next_bike_out"], pred_out))

print(f"MAE  in : {mae_in:.3f} | RMSE in : {rmse_in:.3f}")
print(f"MAE out: {mae_out:.3f} | RMSE out: {rmse_out:.3f}")

-----Training Data-----
         stationid  month  weekhour
817927         114     10       112
952626         136      1        69
3175893        550      1       141
306595          43      9        90
2749116        485      1         7
...            ...    ...       ...
1211128        178      1        17
2633993        468     12        71
1272866        185      7        87
1056972        150      4        85
1875358        344     10       112

[100 rows x 3 columns]
         next_bike_in  next_bike_out
817927            0.0            1.0
952626            0.0            0.0
3175893           0.0            0.0
306595            0.0            0.0
2749116           0.0            0.0
...               ...            ...
1211128           6.0            4.0
2633993           0.0            0.0
1272866           2.0            0.0
1056972           0.0            1.0
1875358           1.0            1.0

[100 rows x 2 columns]
-----Testing Data-----
         stationid  month  we

In [19]:
from joblib import dump, load

keys = ["stationid", "month", "weekhour"]

test_keys = df.loc[X_test.index, keys].reset_index(drop=True)

compare = pd.DataFrame({
    **{k: test_keys[k] for k in keys},
    "in_true":   y_test["next_bike_in"].to_numpy(),
    "in_pred":   pred_in,
    "out_true":   y_test["next_bike_out"].to_numpy(),
    "out_pred":   pred_out,
})

compare = compare.sort_values(keys).reset_index(drop=True)

comparisonPath = Path("/content/drive/MyDrive/Side hustles/DS+X/Comparisons")
comparisonPath.mkdir(parents=True, exist_ok=True)
comparisonOutputPath = comparisonPath / f"bluebikes_xgb_predictions_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
compare.to_csv(comparisonOutputPath, index=False)
print(f"\nSaved prediction comparisons to: {comparisonOutputPath}")

modelPath = Path("/content/drive/MyDrive/Side hustles/DS+X/Models")
modelPath.mkdir(parents=True, exist_ok=True)
modelOutputPath = modelPath / "bluebikes_xgb_multioutput.joblib"
dump(model, modelOutputPath)
print(f"Saved model to: {modelOutputPath}")


Saved prediction comparisons to: /content/drive/MyDrive/Side hustles/DS+X/Comparisons/bluebikes_xgb_predictions_20251026_010932.csv
Saved model to: /content/drive/MyDrive/Side hustles/DS+X/Models/bluebikes_xgb_multioutput.joblib
